In [1]:
import os
import sys
import dotenv
import warnings
from datetime import datetime, timedelta
import re

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import base64
import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'selenium       {selenium.__version__}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
selenium       4.20.0
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [3]:
obb

OpenBB Platform v4.1.7

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.0.4
    - crypto@1.1.5
    - currency@1.1.5
    - derivatives@1.1.5
    - econometrics@1.1.5
    - economy@1.1.5
    - equity@1.1.5
    - etf@1.1.5
    - fixedincome@1.1.5
    - index@1.1.5
    - news@1.1.5
    - quantitative@1.1.5
    - regulators@1.1.5
    - technical@1.1.6

    - alpha_vantage@1.1.5
    - benzinga@1.1.5
    - biztoc@1.1.5
    - cboe@1.1.5
    - ecb@1.1.5
    - federal_reserve@1.1.5
    - finra@1.1.5
    - finviz@1.0.4
    - fmp@1.1.5
    - fred@1.1.5
    - government_us@1.1.5
    - intrinio@1.1.5
    - nasdaq@1.1.6
    - oecd@1.1.5
    - polygon@1.1.5
    - sec@1.1.5
    - seeking_alpha@1.1.5
    - stockgrid@1.1.5
    - tiingo@1.1.5
    - tmx@1.0.2
 

In [4]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [5]:
obb.user

UserSettings

id: 0664186b-1ad5-72d2-8000-4e6bcb08a99d
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'biztoc_api_key': SecretStr('**********'), 'polygon_api_key': SecretStr('**********'), 'tradier_account_type': None, 'intrinio_api_key': None, 'fred_api_key': SecretStr('**********'), 'tradingeconomics_api_key': None, 'tiingo_token': SecretStr('**********'), 'benzinga_api_key': None, 'alpha_vantage_api_key': SecretStr('**********'), 'nasdaq_api_key': None, 'fmp_api_key': SecretStr('**********'), 'tradier_api_key': None}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot

In [6]:
obb.account

/account
    login
    logout
    save
    refresh
    

In [7]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes
obb.account.refresh()

# Logout
# obb.account.logout()

In [8]:
obb.equity

/equity
    /calendar
    /compare
    /darkpool
    /discovery
    /estimates
    /fundamental
    market_snapshots
    /ownership
    /price
    profile
    screener
    search
    /shorts
    

In [9]:
obb.equity.search("Merck", provider="nasdaq").to_df().head(3)


,symbol,name,nasdaq_traded,exchange,etf,round_lot_size,test_issue,cqs_symbol,nasdaq_symbol,next_shares
0,MRK,"Merck & Company, Inc. Common Stock (new)",Y,N,N,100.0,N,MRK,MRK,N


In [10]:
symbol = "MRK"
company = "Merck"

In [11]:
obj = obb.equity.price.performance(symbol)
obj


OBBject

id: 0664186b-20b4-7df0-8000-380e2219e75f
results: [{'symbol': 'MRK', 'one_day': -0.0013, 'wtd': None, 'one_week': 0.02, 'mtd...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [12]:
obj.results


[FinvizPricePerformanceData(symbol=MRK, one_day=-0.0013, wtd=None, one_week=0.02, mtd=None, one_month=0.031, qtd=None, three_month=0.036699999999999997, six_month=0.2458, ytd=0.193, one_year=0.10310000000000001, two_year=None, three_year=None, four_year=None, five_year=None, ten_year=None, max=None, volatility_week=0.013999999999999999, volatility_month=0.0148, price=130.06, volume=5722074.0, average_volume=8210000.000000001, relative_volume=0.7, analyst_recommendation=None, analyst_score=1.52)]

In [13]:
obj.dict()


{'id': '0664186b-20b4-7df0-8000-380e2219e75f',
 'results': [{'symbol': 'MRK',
   'one_day': -0.0013,
   'wtd': None,
   'one_week': 0.02,
   'mtd': None,
   'one_month': 0.031,
   'qtd': None,
   'three_month': 0.036699999999999997,
   'six_month': 0.2458,
   'ytd': 0.193,
   'one_year': 0.10310000000000001,
   'two_year': None,
   'three_year': None,
   'four_year': None,
   'five_year': None,
   'ten_year': None,
   'max': None,
   'volatility_week': 0.013999999999999999,
   'volatility_month': 0.0148,
   'price': 130.06,
   'volume': 5722074.0,
   'average_volume': 8210000.000000001,
   'relative_volume': 0.7,
   'analyst_recommendation': None,
   'analyst_score': 1.52}],
 'provider': 'finviz',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {}},
   'duration': 156694166,
   'route': '/equity/price/performance',
   'timestamp': datetime.datetime(2024, 5,

In [14]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 129.9,
   'bid_size': 800,
   'ask': 130.0,
   'ask_size': 800,
   'last_price': 130.06,
   'open': 130.82,
   'high': 130.88,
   'low': 129.95,
   'volume': 5722627,
   'prev_close': 130.23,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 126.5012,
   'ma_200d': 114.20525,
   'volume_average': 8230932.0,
   'volume_average_10d': 6810530.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'use_cache': True, 'source': 'iex'}},
   'duration': 171768792,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-12T23:19:14.079596'}}}

In [15]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-12,118.00,118.550,116.695,117.14,5233088.0,117.2581,54248
2023-05-15,117.14,117.740,115.490,116.37,5697236.0,116.3517,70438
2023-05-16,115.88,116.760,115.290,116.08,4278281.0,116.2071,60730
2023-05-17,116.37,116.655,113.480,114.76,7210383.0,114.5498,78904
2023-05-18,114.14,115.070,113.330,114.00,6437144.0,114.0864,75952
...,...,...,...,...,...,...,...
2024-05-06,127.18,127.730,126.760,127.57,6419935.0,127.4189,80172
2024-05-07,127.10,130.425,127.070,130.38,6415640.0,129.3458,87045
2024-05-08,130.58,131.510,129.330,129.55,6580073.0,129.7913,74021


In [16]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-12,411.59,413.6400,409.070,413.42,501017,70481548.0,411.1564
2023-05-15,413.01,413.4300,410.230,412.22,427865,54289383.0,412.1201
2023-05-16,410.25,412.8150,410.240,411.86,447536,57705495.0,411.3774
2023-05-17,415.23,415.8550,410.635,412.35,585038,86786957.0,413.4302
2023-05-18,419.23,419.6700,414.670,414.90,672492,97177195.0,417.1828
...,...,...,...,...,...,...,...
2024-05-06,516.57,516.6100,513.300,513.75,444747,47236717.0,514.6357
2024-05-07,517.14,518.5700,516.450,517.56,432286,50977654.0,517.4968
2024-05-08,517.19,517.7400,515.140,515.26,369150,42012599.0,516.9211


In [17]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='nasdaq', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc. - Common Stock')]

In [18]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,56.62,2024-05-10 20:00:01+00:00,56.35,56.8000,56.290,3927323,56.13,...,66.15,54.12,58.7556,60.24095,3127003,1.003363e+11,1772100000,4.61,12.28,2024-05-23 12:00:00+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,76.11,2024-05-10 20:00:02+00:00,75.80,76.2500,75.530,5103040,75.44,...,77.05,48.66,71.6642,63.13665,7082303,1.351759e+11,1776060000,2.39,31.85,2024-07-16 04:00:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,198.77,2024-05-10 20:00:02+00:00,198.54,199.3399,198.270,7529790,197.50,...,200.94,133.96,192.1660,166.00345,8842138,5.708018e+11,2871670000,16.58,11.99,2024-07-12 12:30:00+00:00
3,MS,Morgan Stanley,NYSE,98.28,2024-05-10 20:00:02+00:00,98.74,98.9900,97.625,6118425,98.11,...,98.99,69.42,91.2008,85.73105,8073372,1.597207e+11,1625160000,5.50,17.87,2024-07-16 04:00:00+00:00


In [19]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [20]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,tags,id,amp_url,publisher
date,,,,,,,,,,
2024-05-07 10:20:07+00:00,Should You Invest in the Invesco S&P 500 Equal...,Sector ETF report for RSPH,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2269228/shoul...,"RSPH,MRK,CI,WST,XLV,VHT",Zacks Equity Research,None,fsu82w8dXastE38DzeowYOxzq9FCS7S3msQLXKUJ6YY,https://www.zacks.com/amp/stock/news/2269228/s...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-07 15:30:00+00:00,A Look at Pharma ETFs Post Q1 Earnings,"Many industry bigwigs reported solid results, ...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2269782/a-loo...,"BMY,JNJ,MRK,LLY,PJP,IHE,XPH,PPH,FTXH",Sweta Killa,None,sNchkXpEjItDlY19xKtiys_FBgeVOIoZIEhqQEd_o3g,https://www.zacks.com/amp/stock/news/2269782/a...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-08 06:26:57+00:00,Jim Cramer Advises Investors To Brace For Econ...,"Jim Cramer, the host of CNBC’s “Mad Money,” ha...",[{'url': 'https://cdn.benzinga.com/files/image...,https://www.benzinga.com/news/24/05/38691510/j...,"AAPL,BLDR,GOOG,NVDA,MRK,PFE,GOOGL,META",Benzinga Neuro,"News,Global,Economics,Federal Reserve,Markets",nabJYp9Ewiso_ihbgnuIkdcXtn3Ihy-oucZmO2ALnCk,https://www.benzinga.com/amp/content/38691510,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-09 13:00:16+00:00,"Investors Heavily Search Merck & Co., Inc. (MR...","Recently, Zacks.com users have been paying clo...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2271265/inves...,MRK,Zacks Equity Research,None,LThWA5Sh3t7k8BJ8547R71qJY_9Mgkha7WMQAbZTGro,https://www.zacks.com/amp/stock/news/2271265/i...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-10 15:18:00+00:00,Pharma Stock Roundup: PFE DMD Study Patient De...,Pfizer (PFE) reports the death of a participan...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2272311/pharm...,"PFE,MRK,LLY",Kinjel Shah,None,LuH9atBP810MgCmcYvB9qbOEDI5RlsMVZB1l1s1W8D8,https://www.zacks.com/amp/stock/news/2272311/p...,{'favicon_url': 'https://s3.polygon.io/public/...


# Question Answering Agent

In [21]:
MODEL = "gpt-4-turbo"

MAX_INPUT_TOKENS = 65536     # includes text of all headlines
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0



In [22]:
def get_response(messages, tools, model=MODEL, json_format=False):
    """
    Get a single response from ChatGPT based on a chain of messages.

    Args:
        messages (list): A list of message objects representing the conversation history.
        json_format(boolean): True if JSON response requested. (Last message must express the request for JSON response.)

    Returns:
        dict: A response object containing the generated response from ChatGPT.

    Raises:
        OpenAIError: If there is an error during the API call.

    Example:
        >>> messages = [
        ...     {"role": "system", "content": "You are a helpful assistant."},
        ...     {"role": "user", "content": "What's the weather like today?"},
        ... ]
        >>> response = get_response(messages)
    """

    if tools:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",  # auto is default, but we'll be explicit
            response_format={"type": "json_object"} if json_format else None,
        )
    else:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format={"type": "json_object"} if json_format else None,
        )

    return response


In [23]:
client = OpenAI()
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

The question "What is the airspeed velocity of an unladen swallow?" is a famous line from the movie "Monty Python and the Holy Grail." It's used as a humorous example of dialog based on asking specific but somewhat absurd questions, often without a clearly expected serious answer.

However, if we consider the question from a biological and scientific standpoint, the airspeed velocity of a swallow actually pertains to bird flight speeds. For the European Swallow (often assumed to be the type referenced in the movie), the estimated average cruising speed when migrating has been roughly calculated to be about 20.1 miles per hour or 32.4 kilometers per hour. These figures, however, can vary depending on factors such as wind and carrying load (even though "unladen" implies carrying no load).

The question also sparks interest because it highlights the influence of popular culture in how we often frame and understand scientific queries.


In [24]:
# support use of client-side tools when querying openai

def eval_tool(tool_call, verbose=True):
    """
    Given an OpenAI tool_call response,
    evaluates the tool function using the arguments provided by OpenAI,
    and returns the message to send back to OpenAI, including the function return value.

    Args:
        tool_call (object): The OpenAI tool_call response.

    Returns:
        dict: The message to send back to OpenAI, containing the tool_call_id, role, name, and value returned by the tool call.

    """
    function_name = tool_call.function.name
    # look up the function based in global tools on the name
    fn = tools[function_name]['callable']
    # make the tool call's json args into a dict
    kwargs = json.loads(tool_call.function.arguments)
    
    if verbose:
        print(f"{function_name}({str(kwargs)}) -> ", end="")
    # call function with the args and return value
    fn_value = fn(**kwargs)
    if type(fn_value) is list or type(fn_value) is dict:
        fn_value=str(fn_value)
    if verbose:
        output = str(fn_value)
        if len(output) > 100:
            output=output[:100] + "..."
        print(output)

    return {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": tool_call.function.name,
        "content": fn_value,
    }


def get_response_and_eval(messages, tools=[], json_format=False, verbose=False):
    """
    Sends a list of messages to OpenAI and returns the response.
    If tool calls are returned, calls all the tools and sends the values back to OpenAI.
    If further tool calls returned, iterates until no more tool calls are returned and
    'stop' is returned as finish_reason, then returns the response.

    Args:
        messages (list): A list of messages to send to OpenAI.
        json_format (boolean): If the final response should be in JSON format.
        verbose (bool, optional): If True, prints additional information. Defaults to False.

    Returns:
        response: The final response object returned by OpenAI.

    Raises:
        None

    """
    response = get_response(messages, tools=tools, json_format=json_format)
    choice = response.choices[0]
    response_message = choice.message
    finish_reason = choice.finish_reason

    if verbose:
        print(choice)

    while finish_reason != 'stop':
        # Extend conversation with assistant's reply
        messages.append(response_message)
        if finish_reason == 'tool_calls':
            tool_calls = response_message.tool_calls
            if verbose:
                print(tool_calls)
            # Call the tools and add all return values as messages
            for tool in tool_calls:
                messages.append(eval_tool(tool, verbose=verbose))
            # Get next response
            response = get_response(messages, tools=tools, json_format=json_format)
            choice = response.choices[0]
            response_message = choice.message
            finish_reason = choice.finish_reason
            if verbose:
                output = str(choice)
                output = output[:1000] + "..." if len(output) > 1000 else output
                print(output)
        else:
            print('finish_reason: ', finish_reason)
            break

    return response
import pdb

def agent_query(user_message, tools, verbose=True):
    """
    Send a user message to OpenAI and retrieve the response, calling all tools until done.

    Args:
        user_message (str): The message from the user.
        verbose (bool, optional): Display intermediate tool calls and return values. Defaults to False.

    Returns:
        str: The response from the agent.


    Example:
        >>> agent_query("Hello")
        'Hello! How can I assist you today?'
    """

    # add descriptions of available tools to system prompt
    tool_descs = ""
    tools = list(tools.values())
    openai_tools = []
    for v in tools:
        t = v['tooldict']
        openai_tools.append(t)
        tname = t['function']['name']
        tdesc = t['function']['description']
        tool_descs += f"{tname} : {tdesc}"
        example_str = v.get('example_str')
        if example_str:
            tool_descs += f" Usage: {example_str} "
        tool_descs += "\n---\n\n"
    # tool_descs = "\n".join([f"{tool['function']['name']} : {tool['function']['description']}" for tool in tools.values()])
    current_system_prompt = system_prompt + f"""

Available tools delimited by ---:
{tool_descs}
    """
    # print(current_system_prompt)

    messages = [{"role": "system", "content": current_system_prompt},
                {"role": "user", "content": user_message}]
    response = get_response_and_eval(messages, tools=openai_tools, json_format=False, verbose=verbose)
    response_str = response.choices[0].message.content
    response_str = response_str.replace("$", "\\\$")   # escape stuff that is interpreted as latex
    display(Markdown(response_str))



In [25]:
tools = {}            # tools description sent to openai


In [26]:
# make a tool openai can use
def get_10k_item1_from_symbol(symbol):
    """
    Gets item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    return item1_text

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "example_parameters": [{
        "symbol": "MSFT",
    }],
}

# create tool dict based on metadata
def make_tool_dict(fn_metadata):
    retdict  = {
        "type": "function",
        "function": {
            "name": fn_metadata["name"],
            "description": fn_metadata["description"],
            "parameters": {
                "type": "object",
                "properties": fn_metadata["parameters"],
                "required": [k for k in fn_metadata["parameters"].keys()]
                }
            }
        }
    return retdict

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [27]:
item_1_text = get_10k_item1_from_symbol(symbol)
len(item_1_text)

190265

In [28]:
def create_example_code(fn_metadata):
    example_str = ""
    for example in fn_metadata['example_parameters']:
        if example_str:
            example_str += " ; "
        example_str += f"{fn_metadata['name']}("
        parray = []
        for k, v in example.items():
            # get type 
            ptype = "string"
            if ptype == "string":
                parray.append(f'{k}="{v}"') 
            else:
                parray.append(f'{k}={v}') 
        example_str += ", ".join(parray) 
        example_str += ")" 
        example_str += " -> " 
        return_value = fn_metadata["callable"](**example)
        if type(return_value) is list:
            return_value = str(return_value[:3])
        if type(return_value) is str:
            return_value = return_value[:1000] + "..." if len(return_value) > 1000 else return_value
        example_str += return_value
        return example_str

# create_example_code(fn_metadata)


In [29]:
# example of calling the function on example params
fn_metadata["callable"](**fn_metadata['example_parameters'][0])


"Note About Forward-Looking Statements\nThis report includes estimates, projections, statements relating to our business plans, objectives, and expected operating results that are “forward-looking statements” within the meaning of the Private Securities Litigation Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934. Forward-looking statements may appear throughout this report, including the following sections: “Business” (Part I, Item 1 of this Form 10-K), “Risk Factors” (Part I, Item 1A of this Form 10-K), and “Management’s Discussion and Analysis of Financial Condition and Results of Operations” (Part II, Item 7 of this Form 10-K). These forward-looking statements generally are identified by the words “believe,” “project,” “expect,” “anticipate,” “estimate,” “intend,” “strategy,” “future,” “opportunity,” “plan,” “may,” “should,” “will,” “would,” “will be,” “will continue,” “will likely result,” and similar expressions.

In [30]:
system_prompt = """
Role: You are an AI stock market assistant tasked with providing up-to-date and detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate and concise answers relevant to their questions about individual stocks.

Capabilities: You are given a number of tools as functions. Use as many tools as needed to ensure all information provided is timely, accurate, concise, relevant, and specific to the user's query.

Instructions: First, analyze the query to fully understand the intent of the user, what data they need, and which tools will satisfy their request.
Then, call the most relevant tools in the order necessary to respond accurately to the investor query. 
If the question is not about a specific stock, if no relevant tool is available, or if uncertain or lacking data, say so clearly and suggest alternative resources or data sources that may help.
Only use data obtained from the tools.

Example Interaction:
Investor: What is the current P/E ratio of Apple Inc.?
AI Response: As of the close of trading on March 10, 2024, Apple Inc.'s P/E ratio is 28.45.
"""


In [31]:
tools

{'get_10k_item1_from_symbol': {'name': 'get_10k_item1_from_symbol',
  'description': "Given a stock symbol, retrieves item 1 of the company's latest 10-K annual report filing.",
  'openapi_path': None,
  'callable': <function __main__.get_10k_item1_from_symbol(symbol)>,
  'parameters': {'symbol': {'type': 'string',
    'description': 'The symbol to get the 10-K item 1 for'}},
  'example_parameters': [{'symbol': 'MSFT'}],
  'tooldict': {'type': 'function',
   'function': {'name': 'get_10k_item1_from_symbol',
    'description': "Given a stock symbol, retrieves item 1 of the company's latest 10-K annual report filing.",
    'parameters': {'type': 'object',
     'properties': {'symbol': {'type': 'string',
       'description': 'The symbol to get the 10-K item 1 for'}},
     'required': ['symbol']}}}}}

In [32]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(user_message, tools=tools, verbose=False)


Item 1 from Microsoft's latest 10-K annual report provides an overview of the company's business and future outlook:

1. **Business Overview**: Microsoft identifies itself as a technology company with a mission to empower every person and every organization on the planet to achieve more. The company outlines its commitment to producing innovative platforms and tools, particularly those leveraging AI, to enhance economic outcomes across various sectors including education and public services.

2. **Product and Services**: Microsoft develops, licenses, and supports a wide range of software products, services, and devices. The company offers:
   - **Cloud-based solutions**: Including compute, storage, and database, alongside integrating AI capabilities to enhance functionality.
   - **Productivity applications**: Spanning email, calendaring, and various productivity applications across devices and platforms.
   - **Operating systems**: These are predominantly for managing hardware devices, data storage, applications, and secure communications.
   - **Server applications**: Tailored to support business applications, data analysis, storage, and management.
   - **Business solution applications**: These include Dynamics products for customer relationship management and enterprise resource planning.
   - **Desktop and server management tools**: To streamline management tasks.
   - **Software development tools**: Designed for computer programmers.
   - **Video games**: Developed or published for multiple gaming platforms.
   - **Hardware**: This includes PCs, tablets, gaming and entertainment consoles, and other smart devices.

3. **Focus on AI and Cloud**: Microsoft highlights its strategic investment in AI, infusing AI across its products like Microsoft Teams, Outlook, Bing, and Xbox. The focus is on extending the capabilities of these tools to harness AI for more personalized and efficient user experiences.

4. **Future Plans and Ambitions**: Microsoft emphasizes ongoing innovation in its core areas: reinventing productivity and business processes, building intelligent cloud platforms, and creating more personal computing experiences.

5. **Geographical and Segment Focus**: Microsoft describes its extensive operational scope across different regions, with revenues diversified across its More Personal Computing, Intelligent Cloud, and Productivity and Business Processes segments.

This summary outlines Microsoft's expansive product offerings, strategic focus areas, and broad operational base, underscoring its commitment to innovation, particularly in AI and cloud-based solutions.

In [33]:
# query search using Sec-api
# queryApi = QueryApi(api_key=os.environ['SEC_API_KEY'])

# query = {
#   "query": "ticker:TSLA AND filedAt:[2024-01-01 TO 2024-12-31] AND formType:\"10-K\"",
#   "from": "0",
#   "size": "100",
#   "sort": [{ "filedAt": { "order": "desc" } }]
# }

# response = queryApi.get_filings(query)

# print(response)
# extractorApi = ExtractorApi(os.getenv('SEC_API_KEY'))
# url_10k = 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm'
# item_1_text = extractorApi.get_section(url_10k, "1", "text")
# print(item_1_text)

# 10-K example
# url_10k = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20231231.htm"

# item_1A_text = extractorApi.get_section(url_10k, "1A", "text")
# item_1_text = extractorApi.get_section(url_10k, "1", "text")

# item_7_html = extractorApi.get_section(url_10k, "7", "html")

# 10-Q example
# url_10q = "https://www.sec.gov/Archives/edgar/data/1318605/000095017022006034/tsla-20220331.htm"

# part2_item_1A_text = extractorApi.get_section(url_10q, "part2item1a", "text")

# 8-K example
# url_8k = "https://www.sec.gov/Archives/edgar/data/66600/000149315222016468/form8-k.htm"



# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [34]:
# TODO:

# check additional prompt improvements

# add start_date and end_date for historical queries

# use assistants API


In [35]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]

"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [36]:
# output all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [37]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [38]:
# using the openapi.json OpenAPI definition , output a proposed function to call the openbb function
# might be a better way to do this with standard openapi tools to parse the spec, but didn't find one.

def fn_str(path_str):
    d = data['paths'][path_str]
    # operationId = d['get']['operationId']
    op = path_str.removeprefix("/api/v1/")
    operation = op.replace('/', '.')    
    fn_name = op.replace('/', '_')
    docstring = f"{d['get']['description']}"
    req_params = ", ".join([p['name'] for p in d['get']['parameters'] if p['required']])
    opt_params = ", ".join([p['name'] for p in d['get']['parameters'] if not p['required']])

    retval = f"""
path:                 {path_str}
operation:            {operation}
summary:              {d['get']['summary']}
description:          {d['get']['description']}
required parameters:  {req_params}
optional parameters:  {opt_params}

def get_{fn_name}_json({req_params}):
    \"\"\"{docstring}
    \"\"\"

    retval = None
    try:
        obj = obb.{operation}({req_params})
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval

"""
    return retval

path_str = '/api/v1/equity/search'
print(fn_str(path_str))


path:                 /api/v1/equity/search
operation:            equity.search
summary:              Search
description:          Search for stock symbol, CIK, LEI, or company name.
required parameters:  provider
optional parameters:  query, is_symbol, use_cache, active, limit, is_etf, is_fund

def get_equity_search_json(provider):
    """Search for stock symbol, CIK, LEI, or company name.
    """

    retval = None
    try:
        obj = obb.equity.search(provider)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval




In [39]:
# use metadata to generate tool, example_str
# go through adding example_str to metadata, add the path
# attempt to gen the code and metadata from the path
# fix dates
# add it to tools definitions sent to openai
# fix historical functions with dates, simplify output


def get_equity_search_symbol(search_str):
    """Given a search string, get the stock symbol of the top company whose name best matches the search string.
    """
    retval = None
    try:
        obj = obb.equity.search(search_str)
        retval = obj.results[0].symbol
    except Exception as exc:
        print(exc)
    return retval


fn_metadata = {
    "name": "get_equity_search_symbol",
    "description": "Given a search string, get the stock symbol of the top company whose name best matches the search string.",
    "openapi_path" : '/api/v1/equity/search',
    "callable": get_equity_search_symbol,
    "parameters": {
        "search_str": {
            "type": "string",
            "description": "The search string to match to the stock symbol."
            }
        },
    "example_parameters": [{
        "search_str": "Broadcom",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [40]:
tools['get_equity_search_symbol']

{'name': 'get_equity_search_symbol',
 'description': 'Given a search string, find the stock symbol of the top company whose name best matches the search string.',
 'openapi_path': '/api/v1/equity/search',
 'callable': <function __main__.get_equity_search_symbol(search_str)>,
 'parameters': {'search_str': {'type': 'string',
   'description': 'The search string to find the stock symbol.'}},
 'example_parameters': [{'search_str': 'Broadcom'}],
 'example_str': 'get_equity_search_symbol(search_str="Broadcom") -> AVGO',
 'tooldict': {'type': 'function',
  'function': {'name': 'get_equity_search_symbol',
   'description': 'Given a search string, find the stock symbol of the top company whose name best matches the search string.',
   'parameters': {'type': 'object',
    'properties': {'search_str': {'type': 'string',
      'description': 'The search string to find the stock symbol.'}},
    'required': ['search_str']}}}}

In [41]:
agent_query("What is the stock symbol for Salesforce?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HtKJ8pE1Cd97zHLbnA9dU18A', function=Function(arguments='{"search_str":"Salesforce"}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_HtKJ8pE1Cd97zHLbnA9dU18A', function=Function(arguments='{"search_str":"Salesforce"}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'search_str': 'Salesforce'}) -> CRM
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The stock symbol for Salesforce is CRM.', role='assistant', function_call=None, tool_calls=None))


The stock symbol for Salesforce is CRM.

In [85]:
# first get symbol, then get annual report for symbol 
agent_query("What is item 1 from the 10k annual report for Microsoft?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nxvi2P6a07OrqJB64sWOMhoS', function=Function(arguments='{"search_str":"Microsoft"}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_nxvi2P6a07OrqJB64sWOMhoS', function=Function(arguments='{"search_str":"Microsoft"}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'search_str': 'Microsoft'}) -> MSFT
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_d8KASO74Nc4wvB16o3SrUMXo', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_d8KASO74Nc4wvB16o3SrUMXo', function=Function(arguments='{"symbol":"

Item 1 from Microsoft's latest 10-K annual report discusses several critical aspects of the company:

1. **General Business Overview**:
   - Microsoft describes itself as a technology company whose mission is "to empower every person and every organization on the planet to achieve more." 
   - The company emphasizes its commitment to providing platforms and tools powered by artificial intelligence (AI) to enhance various sectors such as business, education, and public services.

2. **Offerings**:
   - Microsoft develops and supports software, services, devices, and solutions. It offers cloud-based solutions, solution support, consulting services, relevant online advertising, and more.
   - Key products include operating systems, productivity and collaboration applications, server applications, business solutions, and devices like PCs, tablets, and gaming consoles.

3. **Strategic Focus Areas**:
   - The primary ambitions driving the company are: reinventing productivity and business processes, building the intelligent cloud and intelligent edge platform, and creating more personal computing experiences.
   - Specific emphasis is placed on AI and cloud computing enhancements across the organization.

4. **Corporate Social Responsibility and Sustainability**:
   - Microsoft has committed to being carbon negative and zero waste by 2030.
   - The company stresses its efforts to foster sustainability through considerable investments in global environmental initiatives.

The report also elaborates on Microsoft's broad approach to addressing digital inequality, cybersecurity training, and providing pathways for economic and social inclusion through technology. The company acknowledges the challenges posed by rapid technology changes and emphasizes its strategy of continuous innovation to meet these challenges effectively.

This detailed narrative is framed within Microsoft’s broader emphasis on compliance with forward-looking statements legislation, ensuring a comprehensive perspective on risks, future predictions, and market forecasts included in their reporting.

In [43]:
def get_quote_json(symbol):
    """Given a stock symbol, get the latest market data quote for the stock in JSON format.

    Args:
        symbol (str): The symbol of the stock.

    Returns:
        str: The JSON representation of the latest quote data for the given stock.

    Raises:f
        None

    Example:
        >>> get_quote_json('AAPL')
        '{"symbol": "AAPL", "price": 150.25, "volume": 1000, ...}'
    """
    retval = None
    retlist = obb.equity.price.quote(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


fn_metadata = {
    "name": "get_quote_json",
    "description": "Given a stock symbol, get the latest market data quote for the stock in JSON format.",
    "openapi_path" : '/api/v1/equity/price/quote',
    "callable": get_quote_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [44]:
agent_query(f"What is the last market quote for symbol {symbol}?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SFlakN8FCRADa5cF1BlCkU1y', function=Function(arguments='{"symbol":"MRK"}', name='get_quote_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_SFlakN8FCRADa5cF1BlCkU1y', function=Function(arguments='{"symbol":"MRK"}', name='get_quote_json'), type='function')]
get_quote_json({'symbol': 'MRK'}) -> {"symbol":"MRK","asset_type":"stock","name":"MERCK & CO INC COM","exchange":null,"bid":129.9,"bid_si...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The last market quote for Merck & Co Inc (symbol: MRK) as of the close of trading on May 10, 2024, was $129.49.', role='assistant', function_call=None, tool_calls=None))


The last market quote for Merck & Co Inc (symbol: MRK) as of the close of trading on May 10, 2024, was \\$129.49.

In [45]:
def get_company_profile_json(symbol):
    """
    Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format

    Parameters:
        symbol (str): The symbol of the company

    Returns:
        str: The JSON representation of the company's profile information
    """
    retval = None
    retlist = obb.equity.profile(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


fn_metadata = {
    "name": "get_company_profile_json",
    "description": "Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
    "openapi_path" : '/api/v1/equity/profile',
    "callable": get_company_profile_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)



In [46]:
agent_query(f"Can you provide a basic company profile of symbol {symbol}", tools=tools, verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_6cH9hSsMgLGMHMAyMUZOcosu', function=Function(arguments='{"symbol":"MRK"}', name='get_company_profile_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_6cH9hSsMgLGMHMAyMUZOcosu', function=Function(arguments='{"symbol":"MRK"}', name='get_company_profile_json'), type='function')]
get_company_profile_json({'symbol': 'MRK'}) -> {"symbol":"MRK","name":"Merck & Co Inc","cik":null,"cusip":null,"isin":null,"lei":null,"legal_name":...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Merck & Co Inc (Symbol: MRK), located in Rahway, NJ, USA, operates in the healthcare sector, specifically as a drug manufacturer in the general category. The company was founded in 1891 and is listed on the NYSE. Merck is involved in providing health solutions through 

Merck & Co Inc (Symbol: MRK), located in Rahway, NJ, USA, operates in the healthcare sector, specifically as a drug manufacturer in the general category. The company was founded in 1891 and is listed on the NYSE. Merck is involved in providing health solutions through its prescription medicines, vaccines, biologic therapies, animal health, and consumer care products.

The company operates through the following segments:
- **Pharmaceutical**: This includes human health pharmaceutical and vaccine products.
- **Animal Health**: This segment focuses on discovering, developing, manufacturing, and marketing animal health products for livestock and companion animals.
- **Other**: Comprises sales from non-reportable segments of healthcare services.

Merck is a member of both the Dow Jones Industrial Average (DJIA) and the S&P 500 index. As of their last update, they employ 72,000 people and have a market capitalization of approximately \\$329.42 billion. The company has 2.53 billion shares outstanding, with institutional ownership at 78.09%. The latest earnings date was reported as April 25, before market open (BMO), and the stock has a beta value of 0.42, indicating lower volatility relative to the market.

In [47]:
def get_equity_shorts_short_interest(symbol):
    """Given a stock symbol, get data on short volume and days to cover in JSON format.

    Args:
        symbol (str): The stock symbol for which to retrieve short interest data.

    Returns:
        str: The JSON-formatted data containing short volume and days to cover.

    Example:
        >>> get_equity_shorts_short_interest('AAPL')
        '{"short_volume": 10000, "days_to_cover": 5}'
    """
    retval = None
    try:
        obj = obb.equity.shorts.short_interest(symbol)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval


fn_metadata = {
    "name": "get_equity_shorts_short_interest",
    "description": "Given a stock symbol, get data on short volume and days to cover in JSON format.",
    "openapi_path" : '/api/v1/equity/profile',
    "callable": get_equity_shorts_short_interest,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [48]:
agent_query(f"Provide latest statistics on short interest for symbol {symbol}", tools=tools, verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SePVKoOn2tDndO4yEeC3pipv', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_shorts_short_interest'), type='function')]))
[ChatCompletionMessageToolCall(id='call_SePVKoOn2tDndO4yEeC3pipv', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_shorts_short_interest'), type='function')]
get_equity_shorts_short_interest({'symbol': 'MRK'}) -> {"settlement_date":"2021-07-15","symbol":"MRK","issue_name":"Merck & Co., Inc.","market_class":"NYSE...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The latest statistics on short interest for Merck & Co., Inc. (symbol: MRK) as of the settlement date July 15, 2021, are as follows:\n\n- Current short position: 22,245,886 shares\n- Previous short position: 23,932,756 shares\n- Average daily vo

The latest statistics on short interest for Merck & Co., Inc. (symbol: MRK) as of the settlement date July 15, 2021, are as follows:

- Current short position: 22,245,886 shares
- Previous short position: 23,932,756 shares
- Average daily volume: 7,804,274 shares
- Days to cover: 2.85 days
- Change in short position: Decrease of 1,686,870 shares
- Percentage change in short position: -7.05%

In [49]:
def get_equity_fundamental_historical_splits(symbol):
    """Given a stock symbol, get the company's historical stock splits in JSON format.

    Args:
        symbol (str): The stock symbol for the company.

    Returns:
        str: The JSON representation of the company's historical stock splits.

    """
    retval = None
    try:
        obj = obb.equity.fundamental.historical_splits(symbol)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval


fn_metadata = {
    "name": "get_equity_fundamental_historical_splits",
    "description": "Given a stock symbol, get the company's historical stock splits in JSON format.",
    "openapi_path" : '/api/v1/equity/fundamental/historical_splits',
    "callable": get_equity_fundamental_historical_splits,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [50]:
agent_query(f"Provide latest split information for symbol {symbol}", tools=tools, verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_eDs5qmQoGUIGCMJIZ8rEFesk', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_historical_splits'), type='function')]))
[ChatCompletionMessageToolCall(id='call_eDs5qmQoGUIGCMJIZ8rEFesk', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_historical_splits'), type='function')]
get_equity_fundamental_historical_splits({'symbol': 'MRK'}) -> {"date":"1999-02-17","numerator":2.0,"denominator":1.0,"split_ratio":null,"label":"February 17, 99"}
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The latest stock split for Merck & Co., Inc. (symbol MRK) occurred on February 17, 1999. It was a 2-for-1 split.', role='assistant', function_call=None, tool_calls=None))


The latest stock split for Merck & Co., Inc. (symbol MRK) occurred on February 17, 1999. It was a 2-for-1 split.

In [51]:
def get_balance_sheet_json(symbol):
    """Given a symbol, get the latest balance sheet data for the company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The balance sheet data in JSON format.

    """
    retval = None
    retlist = obb.equity.fundamental.balance(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


fn_metadata = {
    "name": "get_balance_sheet_json",
    "description": "Given a stock symbol, get the latest balance sheet data for the company in JSON format.",
    "openapi_path" : '/api/v1/equity/fundamental/balance',
    "callable": get_balance_sheet_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [52]:
agent_query(f"what are the latest total assets for symbol {symbol}", tools=tools, verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_RYV7mDs9BWOVJDIPfGngT0J1', function=Function(arguments='{"symbol":"MRK"}', name='get_balance_sheet_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_RYV7mDs9BWOVJDIPfGngT0J1', function=Function(arguments='{"symbol":"MRK"}', name='get_balance_sheet_json'), type='function')]
get_balance_sheet_json({'symbol': 'MRK'}) -> {"period_ending":"2023-12-31","fiscal_period":"FY","fiscal_year":2023,"filing_date":"2024-04-11","ac...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='As of the fiscal year ending December 31, 2023, Merck & Co. Inc. (symbol: MRK) reported total assets of $106.675 billion.', role='assistant', function_call=None, tool_calls=None))


As of the fiscal year ending December 31, 2023, Merck & Co. Inc. (symbol: MRK) reported total assets of \\$106.675 billion.

In [53]:
# def get_balance_sheet_growth_json(symbol):
#     """Given a stock symbol, get the percent changes in the most recent balance sheet data for the company in JSON format.

#     Args:
#         symbol (str): The symbol of the company.

#     Returns:
#         str: The JSON representation of the balance sheet growth.

#     """
#     retval = None
#     retlist = obb.equity.fundamental.balance_growth(symbol).results
#     if retlist and type(retlist is list):
#         retval = retlist[0].model_dump_json()
#     return retval


# fn_metadata = {
#     "name": "get_balance_sheet_growth_json",
#     "description": "Given a stock symbol, get the percent changes in the most recent balance sheet data for the company in JSON format.",
#     "openapi_path" : '/api/v1/equity/fundamental/balance_growth',
#     "callable": get_balance_sheet_growth_json,
#     "parameters": {
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     "example_parameters": [{
#         "symbol": "NVDA",
#     }],
# }

# tools[fn_metadata["name"]] = fn_metadata
# tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
# tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [54]:
# agent_query(f"what was the most recent percentage growth in total assets for symbol {symbol}", tools=tools, verbose=True)

In [55]:
def get_cash_flow_json(symbol):
    """Given a stock symbol, get the latest cash flow statement data for the company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The cash flow statement data in JSON format.

    """
    retval = None
    retlist = obb.equity.fundamental.cash(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


fn_metadata = {
    "name": "get_cash_flow_json",
    "description": "Given a stock symbol, get the latest cash flow statement data for the company in JSON format.",
    "openapi_path" : '/api/v1/equity/fundamental/cash',
    "callable": get_cash_flow_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}


tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [56]:
agent_query(f"what was the most recent cash flow from operations for symbol {symbol}", tools=tools, verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hF52Hq3xrFBWciUhGQqvipB0', function=Function(arguments='{"symbol":"MRK"}', name='get_cash_flow_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_hF52Hq3xrFBWciUhGQqvipB0', function=Function(arguments='{"symbol":"MRK"}', name='get_cash_flow_json'), type='function')]
get_cash_flow_json({'symbol': 'MRK'}) -> {"period_ending":"2023-12-31","fiscal_period":"FY","fiscal_year":2023,"filing_date":"2024-04-11","ac...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The most recent cash flow from operations for Merck & Co, Inc. (symbol MRK) for the fiscal year ending December 31, 2023, was $13.006 billion USD.', role='assistant', function_call=None, tool_calls=None))


The most recent cash flow from operations for Merck & Co, Inc. (symbol MRK) for the fiscal year ending December 31, 2023, was \\$13.006 billion USD.

In [57]:
# def get_cash_flow_growth_json(symbol):
#     """Given a stock symbol, get the changes in the company's cash flow statement items in JSON format.

#     Args:
#         symbol (str): The symbol of the company.

#     Returns:
#         str: The JSON representation of the growth of the company's cash flow statement items in JSON format.

#     """
#     retval = None
#     retlist = obb.equity.fundamental.cash_growth(symbol).results
#     if retlist and type(retlist is list):
#         retval = retlist[0].model_dump_json()
#     return retval


# fn_metadata = {
#     "name": "get_cash_flow_growth_json",
#     "description": "Given a stock symbol, get the percent changes in the company's latest cash flow statement items in JSON format.",
#     "openapi_path" : '/api/v1/equity/fundamental/cash_growth',
#     "callable": get_cash_flow_growth_json,
#     "parameters": {
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     "example_parameters": [{
#         "symbol": "NVDA",
#     }],
# }


# tools[fn_metadata["name"]] = fn_metadata
# tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
# tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [58]:
# agent_query(f"what was the most recent change in cash flow from operations for symbol {symbol}", tools=tools, verbose=True)

In [59]:
def get_income_statement_json(symbol):
    """Given a stock symbol, get the latest income statement data for the company in JSON format

    Args:
        symbol (str): The stock symbol of the company.

    Returns:
        str: The income statement data in JSON format.

    """
    retval = None
    retlist = obb.equity.fundamental.income(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


fn_metadata = {
    "name": "get_income_statement_json",
    "description": "Given a stock symbol, get the latest income statement data for the company in JSON format",
    "openapi_path" : '/api/v1/equity/fundamental/income',
    "callable": get_income_statement_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}


tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [60]:
agent_query(f"what was the most recent net income for symbol {symbol}", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wUuH1z76qkK75hM0Zg1chKFV', function=Function(arguments='{"symbol":"MRK"}', name='get_income_statement_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_wUuH1z76qkK75hM0Zg1chKFV', function=Function(arguments='{"symbol":"MRK"}', name='get_income_statement_json'), type='function')]
get_income_statement_json({'symbol': 'MRK'}) -> {"period_ending":"2023-12-31","fiscal_period":"FY","fiscal_year":2023,"filing_date":"2024-04-11","ac...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The most recent net income for Merck & Co., Inc. (symbol: MRK) for the fiscal year ending December 31, 2023, was $365 million USD.', role='assistant', function_call=None, tool_calls=None))


The most recent net income for Merck & Co., Inc. (symbol: MRK) for the fiscal year ending December 31, 2023, was \\$365 million USD.

In [61]:
# def get_income_growth_json(symbol):
#     """Given a stock symbol, get the percent changes in the company's latest income statement items in JSON format.

#     Args:
#         symbol (str): The stock symbol for the company.

#     Returns:
#         str: The JSON representation of the changes in the company's latest income statement items.

#     Raises:
#         None

#     Examples:
#         >>> get_income_growth_json('AAPL')
#         '{"item1": 100, "item2": 200, "item3": 300}'

#     """
#     retval = None
#     retlist = obb.equity.fundamental.income_growth(symbol).results
#     if retlist and type(retlist is list):
#         retval = retlist[0].model_dump_json()
#     return retval


# fn_metadata = {
#     "name": "get_income_growth_json",
#     "description": "Given a stock symbol, get the percent changes in the company's latest income statement items in JSON format.",
#     "openapi_path" : '/api/v1/equity/fundamental/income_growth',
#     "callable": get_income_growth_json,
#     "parameters": {
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     "example_parameters": [{
#         "symbol": "NVDA",
#     }],
# }


# tools[fn_metadata["name"]] = fn_metadata
# tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
# tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [62]:
# agent_query(f"what was the most recent change in net income for {company}", tools=tools, verbose=True)

In [63]:
def get_fundamental_metrics_json(symbol):
    """Given a stock symbol, get fundamental metrics for the company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The JSON representation of the fundamental metrics.

    """
    retval = None
    retlist = obb.equity.fundamental.metrics(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


fn_metadata = {
    "name": "get_fundamental_metrics_json",
    "description": "Given a stock symbol, get fundamental metrics for the company in JSON format.",
    "openapi_path" : '/api/v1/equity/fundamental/metrics',
    "callable": get_fundamental_metrics_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [64]:
agent_query(f"what was the most recent PE ratio for symbol {symbol}", tools=tools, verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_akxVwO89UcRzK1QelZ0tPqDD', function=Function(arguments='{"symbol":"MRK"}', name='get_fundamental_metrics_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_akxVwO89UcRzK1QelZ0tPqDD', function=Function(arguments='{"symbol":"MRK"}', name='get_fundamental_metrics_json'), type='function')]
get_fundamental_metrics_json({'symbol': 'MRK'}) -> {"symbol":"MRK","market_cap":329420000000.0,"pe_ratio":144.78,"foward_pe":13.14,"eps":0.9,"price_to_...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="As of the most recent data, Merck & Co., Inc. (MRK)'s P/E (price-to-earnings) ratio is 144.78.", role='assistant', function_call=None, tool_calls=None))


As of the most recent data, Merck & Co., Inc. (MRK)'s P/E (price-to-earnings) ratio is 144.78.

In [65]:
def get_fundamental_ratios_json(symbol):
    """Given a stock symbol, get fundamental valuation ratios for the company in JSON format.

    Args:
        symbol (str): The stock symbol for the company.

    Returns:
        str: The fundamental valuation ratios for the company in JSON format.

    """
    retval = None
    retlist = obb.equity.fundamental.ratios(symbol).results
    if retlist and type(retlist is list):
        retval = retlist[0].model_dump_json()
    return retval


fn_metadata = {
    "name": "get_fundamental_ratios_json",
    "description": "Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
    "openapi_path" : '/api/v1/equity/fundamental/ratios',
    "callable": get_fundamental_ratios_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [66]:
agent_query(f"what was the most recent price to sales ratio for symbol {symbol}", tools=tools, verbose=True)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mpERngdfMVfvz3fHVeBFz4Ds', function=Function(arguments='{"symbol":"MRK"}', name='get_fundamental_metrics_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_mpERngdfMVfvz3fHVeBFz4Ds', function=Function(arguments='{"symbol":"MRK"}', name='get_fundamental_metrics_json'), type='function')]
get_fundamental_metrics_json({'symbol': 'MRK'}) -> {"symbol":"MRK","market_cap":329420000000.0,"pe_ratio":144.78,"foward_pe":13.14,"eps":0.9,"price_to_...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The most recent price to sales (P/S) ratio for Merck & Co., Inc. (symbol: MRK) is 5.39.', role='assistant', function_call=None, tool_calls=None))


The most recent price to sales (P/S) ratio for Merck & Co., Inc. (symbol: MRK) is 5.39.

In [67]:
def get_equity_fundamental_multiples(symbol):
    """Given a stock symbol, get fundamental valuation multiples for the company in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        str: The JSON representation of the valuation multiples.

    """
    retval = None
    try:
        obj = obb.equity.fundamental.multiples(symbol)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval


fn_metadata = {
    "name": "get_equity_fundamental_multiples",
    "description": "Given a stock symbol, get fundamental valuation multiples for the company in JSON format.",
    "openapi_path" : '/api/v1/equity/fundamental/multiples',
    "callable": get_equity_fundamental_multiples,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [68]:
agent_query(f"what was the most recent revenue per share for symbol {symbol}", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_j20gqltWfQRgWSU4bqbynCmX', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_multiples'), type='function')]))
[ChatCompletionMessageToolCall(id='call_j20gqltWfQRgWSU4bqbynCmX', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_multiples'), type='function')]
get_equity_fundamental_multiples({'symbol': 'MRK'}) -> {"symbol":"MRK","revenue_per_share_ttm":24.25424397947098,"net_income_per_share_ttm":0.9103829451243...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The most recent revenue per share for Merck & Co., Inc. (symbol: MRK) on a trailing twelve-month (TTM) basis is $24.25.', role='assistant', function_call=None, tool_calls=None))


The most recent revenue per share for Merck & Co., Inc. (symbol: MRK) on a trailing twelve-month (TTM) basis is \\$24.25.

In [69]:
def get_historical_eps(symbol):
    """Given a stock symbol, get historical earnings per share data for the company in JSON format.
    """
    retval = None
    try:
        obj = obb.equity.fundamental.historical_eps(symbol)
        d = json.loads(obj.json())
        retval = d['results']
    except Exception as exc:
        print(exc)
    return retval


fn_metadata = {
    "name": "get_historical_eps",
    "description": "Given a stock symbol, get historical earnings per share data for the company in JSON format.",
    "openapi_path" : '/api/v1/equity/fundamental/historical_eps',
    "callable": get_historical_eps,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [70]:
agent_query(f"what was the EPS for symbol {company} in the quarter ended 2022-09-30?", tools=tools)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_u0IiRXS6mXGmbr8yKtHQMo0m', function=Function(arguments='{"search_str":"Merck"}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_u0IiRXS6mXGmbr8yKtHQMo0m', function=Function(arguments='{"search_str":"Merck"}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'search_str': 'Merck'}) -> MRK
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_U2VTkQnOD4bTy4vQkCkYLkmx', function=Function(arguments='{"symbol":"MRK"}', name='get_historical_eps'), type='function')]))
[ChatCompletionMessageToolCall(id='call_U2VTkQnOD4bTy4vQkCkYLkmx', function=Function(arguments='{"symbol":"MRK"}', name='get_his

For Merck & Co., Inc. (symbol: MRK), the earnings per share (EPS) for the quarter ended on 2022-09-30 were \\$1.85, which surpassed the estimated EPS of \\$1.71.

In [71]:
def get_equity_fundamental_dividend(symbol):
    """Given a stocek symbol, get the latest dividend data for the company in JSON format.

    Args:
        symbol (str): The symbol of the dcompany.

    Returns:
        str: The JSON representation of the last dividend data.

    """
    retval = None
    try:
        obj = obb.equity.fundamental.dividends(symbol)
        retval = obj.results[0].model_dump_json()
    except Exception as exc:
        print(exc)
    return retval


fn_metadata = {
    "name": "get_equity_fundamental_dividend",
    "description": "Given a stock symbol, get the latest dividend data for the company in JSON format.",
    "openapi_path" : '/api/v1/equity/fundamental/dividends',
    "callable": get_equity_fundamental_dividend,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [72]:
agent_query(f"what was the latest dividend data for symbol {symbol}?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_L7az7q6Y2JZwaO4Jv4SR9fMX', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_dividend'), type='function')]))
[ChatCompletionMessageToolCall(id='call_L7az7q6Y2JZwaO4Jv4SR9fMX', function=Function(arguments='{"symbol":"MRK"}', name='get_equity_fundamental_dividend'), type='function')]
get_equity_fundamental_dividend({'symbol': 'MRK'}) -> {"ex_dividend_date":"2024-03-14","amount":0.77,"label":"March 14, 24","adj_dividend":0.77,"record_da...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The latest dividend data for Merck & Co., Inc. (symbol: MRK) is as follows:\n- Ex-Dividend Date: March 14, 2024\n- Amount: $0.77 per share\n- Adjusted Dividend: $0.77 per share\n- Record Date: March 15, 2024\n- Payment Date: April 5, 2024\n- Declar

The latest dividend data for Merck & Co., Inc. (symbol: MRK) is as follows:
- Ex-Dividend Date: March 14, 2024
- Amount: \\$0.77 per share
- Adjusted Dividend: \\$0.77 per share
- Record Date: March 15, 2024
- Payment Date: April 5, 2024
- Declaration Date: January 23, 2024

In [73]:
def get_trailing_dividend_yield_json(symbol):
    """Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.

    Args:
        symbol (str): The symbol of the company.

    Returns:
        dict: A dictionary containing the 1 year trailing dividend yield for the given company over time.

    Raises:
        Exception: If there is an error retrieving the dividend yield.

    """
    retval = None
    try:
        obj = obb.equity.fundamental.trailing_dividend_yield(symbol=symbol, year=2023)
        d = json.loads(obj.json())
        retval = d['results']
    except Exception as exc:
        print(exc)
    return retval


fn_metadata = {
    "name": "get_trailing_dividend_yield_json",
    "description": "Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
    "openapi_path" : '/api/v1/equity/fundamental/trailing_dividend_yield',
    "callable": get_trailing_dividend_yield_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [74]:
agent_query(f"what was the dividend yield for symbol {symbol} as of the end of 2023?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content="To provide you with the information on the dividend yield for the stock symbol MRK (Merck & Co., Inc.) as of the end of 2023, I'll need to retrieve the trailing dividend yield historical data. Let me fetch that information for you.", role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZEudnC63wKUfghFuQs294CUo', function=Function(arguments='{"symbol":"MRK"}', name='get_trailing_dividend_yield_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_ZEudnC63wKUfghFuQs294CUo', function=Function(arguments='{"symbol":"MRK"}', name='get_trailing_dividend_yield_json'), type='function')]
get_trailing_dividend_yield_json({'symbol': 'MRK'}) -> [{'date': '2023-05-11', 'trailing_dividend_yield': 0.0241599319}, {'date': '2023-05-12', 'trailing_d...
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='As of the end of

As of the end of 2023, the trailing dividend yield for Merck & Co., Inc. (MRK) on December 29, 2023, was approximately 2.72%.

In [75]:
# TODAY = datetime.today().strftime('%Y-%m-%d')
# YESTERDAY = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

# TODAY, YESTERDAY


In [76]:
# def get_price_historical_json(symbol, start_date=YESTERDAY, end_date=TODAY):
#     """
#     Retrieve the historical price data for a given symbol within a specified date range in JSON format.

#     Args:
#         symbol (str): The symbol of the equity.
#         start_date (str, optional): The start date of the historical data. Defaults to YESTERDAY.
#         end_date (str, optional): The end date of the historical data. Defaults to TODAY.

#     Returns:
#         dict: The historical price data in JSON format.

#     """
#     retval = None
#     try:
#         obj = obb.equity.price.historical(symbol=symbol, start_date=start_date, end_date=end_date)
#         retval = obj.json()
#     except Exception as exc:
#         print(exc)
#     return retval


# fn_metadata = {
#     "name": "get_price_historical_json",
#     "description": "Given a stock symbol, get historical price data for a the stock within a specified date range in JSON format.",
#     "openapi_path" : '/api/v1/equity/price/historical',
#     "callable": get_price_historical_json,
#     "parameters": {
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     "example_parameters": [{
#         "symbol": "NVDA",
#     }],
# }

# tools[fn_metadata["name"]] = fn_metadata
# tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
# tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [77]:
def get_price_performance_json(symbol):
    """Given a stock symbol, get price performance data for the stock for different time periods in JSON format."""
    retval = None
    try:
        obj = obb.equity.price.performance(symbol=symbol)
        retval = obj.json()
    except Exception as exc:
        print(exc)
    return retval



fn_metadata = {
    "name": "get_price_performance_json",
    "description": "Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
    "openapi_path" : '/api/v1/equity/price/performance',
    "callable": get_price_performance_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


In [78]:
agent_query(f"what was the performance for {company} from 1 year ago?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XU9e9vFVHtAAg9BosYiPIWpU', function=Function(arguments='{"search_str":"Merck"}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_XU9e9vFVHtAAg9BosYiPIWpU', function=Function(arguments='{"search_str":"Merck"}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'search_str': 'Merck'}) -> MRK
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PBsOnpQcgixIgUMweXVnTjNh', function=Function(arguments='{"symbol":"MRK"}', name='get_price_performance_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_PBsOnpQcgixIgUMweXVnTjNh', function=Function(arguments='{"symbol":"MRK"}', name=

Merck's performance one year ago shows a return of approximately 10.31%. This indicates how the company's stock price has changed over the past year relative to its price one year ago.

In [79]:
def get_etf_equity_exposure_json(symbol):
    """Given a stock symbol, get the ETFs with exposure to the stock in JSON format.

    Args:
        symbol (str): The symbol of the stock.

    Returns:
        dict: A dictionary containing the ETFs with exposure to the specified stock.

    Raises:
        Exception: If an error occurs while retrieving the ETFs.

    """
    retval = None
    try:
        obj = obb.etf.equity_exposure(symbol)
        d = json.loads(obj.json())
        retval = d['results']
    except Exception as exc:
        print(exc)
    return retval


fn_metadata = {
    "name": "get_etf_equity_exposure_json",
    "description": "Given a stock symbol, get the exposure of ETFs to the stock in JSON format.",
    "openapi_path" : '/api/v1/etf/equity_exposure',
    "callable": get_etf_equity_exposure_json,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    "example_parameters": [{
        "symbol": "NVDA",
    }],
}

tools[fn_metadata["name"]] = fn_metadata
tools[fn_metadata["name"]]["example_str"] = create_example_code(fn_metadata)
tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)



In [80]:
pd.DataFrame(get_etf_equity_exposure_json('MRK')).sort_values('weight', ascending=False)

,equity_symbol,etf_symbol,shares,weight,market_value
435,MRK,FTXH,11684.0,0.075700,1.519621e+06
77,MRK,QDVG.DE,1117567.0,0.064076,1.466248e+08
78,MRK,IUHE.AS,0.0,0.062564,1.460573e+08
79,MRK,IUHC.L,1121581.0,0.062374,1.453008e+08
13,MRK,XLV,18454434.0,0.062123,2.400184e+09
...,...,...,...,...,...
563,MRK,GXUS,2163.0,NaN,3.629300e+05
665,MRK,GPAL,640.0,NaN,6.631040e+04
676,MRK,GDEF,397.0,NaN,5.170131e+04
682,MRK,EUNU.DE,50000.0,NaN,3.868500e+04


In [81]:
agent_query(f"which ETF has the highest weight in {company} ?", tools=tools, verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8ZHhhVgK4MFs2kXuIPWz86nx', function=Function(arguments='{"search_str":"Merck"}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_8ZHhhVgK4MFs2kXuIPWz86nx', function=Function(arguments='{"search_str":"Merck"}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'search_str': 'Merck'}) -> MRK
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5O4ILjjsSx92dWzwrTlGgklW', function=Function(arguments='{"symbol":"MRK"}', name='get_etf_equity_exposure_json'), type='function')]))
[ChatCompletionMessageToolCall(id='call_5O4ILjjsSx92dWzwrTlGgklW', function=Function(arguments='{"symbol":"MRK"}', nam

The ETF with the highest weight in Merck (MRK) is the "XLV" with a weight of approximately 0.0621.

In [82]:
tools

{'get_10k_item1_from_symbol': {'name': 'get_10k_item1_from_symbol',
  'description': "Given a stock symbol, retrieves item 1 of the company's latest 10-K annual report filing.",
  'openapi_path': None,
  'callable': <function __main__.get_10k_item1_from_symbol(symbol)>,
  'parameters': {'symbol': {'type': 'string',
    'description': 'The symbol to get the 10-K item 1 for'}},
  'example_parameters': [{'symbol': 'MSFT'}],
  'tooldict': {'type': 'function',
   'function': {'name': 'get_10k_item1_from_symbol',
    'description': "Given a stock symbol, retrieves item 1 of the company's latest 10-K annual report filing.",
    'parameters': {'type': 'object',
     'properties': {'symbol': {'type': 'string',
       'description': 'The symbol to get the 10-K item 1 for'}},
     'required': ['symbol']}}}},
 'get_equity_search_symbol': {'name': 'get_equity_search_symbol',
  'description': 'Given a search string, find the stock symbol of the top company whose name best matches the search string.'

In [83]:
list(tools.values())

[{'name': 'get_10k_item1_from_symbol',
  'description': "Given a stock symbol, retrieves item 1 of the company's latest 10-K annual report filing.",
  'openapi_path': None,
  'callable': <function __main__.get_10k_item1_from_symbol(symbol)>,
  'parameters': {'symbol': {'type': 'string',
    'description': 'The symbol to get the 10-K item 1 for'}},
  'example_parameters': [{'symbol': 'MSFT'}],
  'tooldict': {'type': 'function',
   'function': {'name': 'get_10k_item1_from_symbol',
    'description': "Given a stock symbol, retrieves item 1 of the company's latest 10-K annual report filing.",
    'parameters': {'type': 'object',
     'properties': {'symbol': {'type': 'string',
       'description': 'The symbol to get the 10-K item 1 for'}},
     'required': ['symbol']}}}},
 {'name': 'get_equity_search_symbol',
  'description': 'Given a search string, find the stock symbol of the top company whose name best matches the search string.',
  'openapi_path': '/api/v1/equity/search',
  'callable'

In [84]:
[{t['description']} for t in tools.values()]

[{"Given a stock symbol, retrieves item 1 of the company's latest 10-K annual report filing."},
 {'Given a search string, find the stock symbol of the top company whose name best matches the search string.'},
 {'Given a stock symbol, get the latest market data quote for the stock in JSON format.'},
 {'Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format'},
 {'Given a stock symbol, get data on short volume and days to cover in JSON format.'},
 {"Given a stock symbol, get the company's historical stock splits in JSON format."},
 {'Given a stock symbol, get the latest balance sheet data for the company in JSON format.'},
 {'Given a stock symbol, get the latest cash flow statement data for the company in JSON format.'},
 {'Given a stock symbol, get the latest income statement data for the company in JSON format'},
 {'Given a stock symbol, get fundamental metrics for the company in JSON format.'},
 {'Given a stock